## IMPORTS

In [ ]:
!pip install datasets transformers
# ==4.28.0
!pip install scikit-multilearn
!pip install accelerate -U
!pip install tensorflow
%env SM_FRAMEWORK=tensorflow.keras
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.6 MB/s eta 0:00:00
env: SM_FRAMEWORK=tensorflow.keras


In [ ]:
!pip install nltk
!pip install torch

In [ ]:
import pickle
import pandas as pd
import numpy as np
import nltk
import torch
import string
import os
import tensorflow as tf
import time
from time import sleep
from string import punctuation
from sklearn import metrics
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TFBertModel
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

## Load Data

In [ ]:
long_df = pd.read_csv('long_sypnosis.csv').drop(columns = "Unnamed: 0")

In [ ]:
corrupted_list = []

for img in tqdm(range(long_df.shape[0])):
    try:
        images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{long_df['imdb_id'][img]}.jpg"
        image = Image.open(images_path)
    except:
        corrupted_list.append(long_df['imdb_id'][img])

100%|█████████████████████████████████████████████████████████████████████████████| 4624/4624 [00:02<00:00, 1812.31it/s]


In [ ]:
len(corrupted_list)

681

In [ ]:
long_df = long_df.loc[~long_df['imdb_id'].isin(corrupted_list)]

In [ ]:
long_df = long_df.reset_index(drop=True)

## Preprocess Data
images -> resize and scale
text -> word_ids?

In [ ]:
long_df["genre"] = (
    long_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [ ]:
long_df.head()

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [ ]:
def cleaning(sentence):

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    return sentence

In [ ]:
long_df['sypnosis'] = long_df['sypnosis'].apply(cleaning)

In [ ]:
long_df.head()

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,an anime adaptation of the hindu epic the rama...
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,when the menace known as the joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,gandalf and aragorn lead the world of men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,miles morales catapults across the multiverse ...
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,the story of prophet muhammad and the delivery...


In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

mlb = MultiLabelBinarizer()

def balanced_split(df, mlb, test_size=0.5):
    ind = np.expand_dims(np.arange(len(df)), axis=1)
    mlb.fit_transform(df["genre"])
    labels = mlb.transform(df["genre"])
    ind_train, _, ind_test, _ = iterative_train_test_split(
        ind, labels, test_size
    )
    return df.iloc[ind_train[:, 0]], df.iloc[ind_test[:, 0]]

In [ ]:
df_train, df_test = balanced_split(long_df, mlb, test_size=0.2)
# df_val, df_test = balanced_split(df_tmp, test_size=0.5)

In [ ]:
df_val, df_test = balanced_split(df_test, mlb, test_size=0.5)

In [ ]:
df_train.shape

(3136, 5)

In [ ]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [ ]:
# df_train.to_csv('/content/drive/MyDrive/stack_train.csv')
df_test.to_csv('/content/drive/MyDrive/stack_test_new.csv')
df_val.to_csv('/content/drive/MyDrive/stack_val_new.csv')

In [ ]:
train_imgs = []

for img in tqdm(range(df_train.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_train['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((224, 224))
    train_imgs.append(np.array(image))
    time.sleep(1)
    np.save('train_img', train_imgs)

100%|█████████████████████████████████████████████████████████████████████████████| 3136/3136 [1:07:23<00:00,  1.29s/it]


In [ ]:
train_imgs = np.array(train_imgs)
np.save('train_imgs', train_imgs)

In [ ]:
test_imgs = []

for img in tqdm(range(df_test.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_test['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((224, 224))
    test_imgs.append(np.array(image))
    time.sleep(1)

test_imgs = np.array(test_imgs)
np.save('/content/drive/MyDrive/test_imgs_new', test_imgs)

  0%|          | 0/402 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [ ]:
val_imgs = []

for img in tqdm(range(df_val.shape[0])):
    images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/notebooks/merged/{df_val['imdb_id'][img]}.jpg"
    image = Image.open(images_path)
    image = image.resize((224, 224))
    val_imgs.append(np.array(image))
    time.sleep(1)

val_imgs = np.array(val_imgs)
np.save('/content/drive/MyDrive/val_imgs_new', val_imgs)

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train['genre'])

#transform target variable
y_train = multilabel_binarizer.transform(df_train['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_train[f"{genre_names[i]}"] = y_train[:,i]

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_test['genre'])

#transform target variable
y_test = multilabel_binarizer.transform(df_test['genre'])
genre_names = multilabel_binarizer.classes_

# Adding
for i in range(len(genre_names)):
    df_test[f"{genre_names[i]}"] = y_test[:,i]

In [ ]:
np.save('y_train_stack', y_train)
np.save('y_test_stack', y_test)

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/stack_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/stack_test.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_text = df_train['sypnosis'].to_list()

In [ ]:
len(train_text)

3136

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

test_text = df_test['sypnosis'].to_list()

In [ ]:
len(test_text)

807

In [ ]:
# train_encodings_list = []
# test_encodings_list = []

# for movie in train_text:
#   train_encodings = tokenizer(str(movie), truncation=True, padding=True, max_length=128)
#   train_encodings_list.append(train_encodings['input_ids'])

# for movie in test_text:
#   test_encodings = tokenizer(str(movie), truncation=True, padding=True, max_length=128)
#   test_encodings_list.append(test_encodings['input_ids'])

# model.fit(train_encodings, tf.convert_to_tensor(y_train), epochs=3)

In [ ]:
train_encodings = tokenizer(train_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")
test_encodings = tokenizer(test_text, truncation = True, padding = True, max_length = 128, return_tensors = "np")

In [ ]:
train_encodings['input_ids'].shape

(3136, 128)

In [ ]:
train_encodings.shape

(3136,)

In [ ]:
with open("X_test_text", "wb") as fp:   #Pickling
    pickle.dump(test_encodings_list, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
# ...
# >>> b
# [1, 2, 3, 4]

In [ ]:
with open("X_train_text", "wb") as fp:   #Pickling
    pickle.dump(train_encodings_list, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
# ...
# >>> b
# [1, 2, 3, 4]

In [ ]:
test_text = test_text['input_ids']

In [ ]:
train_text = train_text['input_ids']

In [ ]:
train_imgs = np.load('/content/drive/MyDrive/train_imgs.npy')
test_imgs = np.load('/content/drive/MyDrive/test_imgs.npy')

In [ ]:
y_train = np.load('y_train_stack.npy')
y_test = np.load('y_test_stack.npy')

In [ ]:
y_test.shape

(807, 21)

In [ ]:
# load the dataset and split into train/test sets
# REMEMBER TO DROP COLUMNS!!
X_train_img = train_imgs
X_train_text = train_encodings['input_ids']
y_train = y_train
X_test_img = test_imgs
X_test_text = test_encodings['input_ids']
y_test = y_test
X_val_img = val_imgs
X_val_text = val_encodings['input_ids']
y_val = y_val

In [ ]:
train_imgs[0].shape

for img in train_imgs:
  if img.shape != (224, 224, 3):
    print("wrong dimensions")

## CNN Model

In [ ]:
# define the CNN-based image feature extractor
def build_image_model():
 img_model = tf.keras.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu')
 ])
 return img_model

## BERT Model

In [ ]:
# define the BERT-based text feature extractor
def build_text_model():
 bert_model = TFBertModel.from_pretrained('bert-base-uncased')
 inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_word_ids')
 outputs = bert_model(inputs)[1]
 text_model = tf.keras.Model(inputs=inputs, outputs=outputs)
 return text_model

## Multimodal Model

In [ ]:
# define the multimodal document classification model
def build_multimodal_model(num_classes):
 img_model = build_image_model()
 text_model = build_text_model()
 img_input = tf.keras.layers.Input(shape=(224, 224, 3), name='img_input')
 text_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='text_input')
 img_features = img_model(img_input)
 text_features = text_model(text_input)
 concat_features = tf.keras.layers.concatenate([img_features, text_features])
 x = tf.keras.layers.Dense(512, activation='relu')(concat_features)
 x = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
 multimodal_model = tf.keras.Model(inputs=[img_input, text_input], outputs=x)
 return multimodal_model

In [ ]:
# build the multimodal model
num_classes = 21
multimodal_model = build_multimodal_model(num_classes)
multimodal_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_input (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 text_input (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 sequential (Sequential)        (None, 512)          9678528     ['img_input[0][0]']              
                                                                                                  
 model (Functional)             (None, 768)          109482240   ['text_input[0][0]']       

In [ ]:
# compile the model and train on the train set
multimodal_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'binary_accuracy', 'categorical_accuracy'])

multimodal_model.fit([(X_train_img, X_train_text)], tf.convert_to_tensor(y_train), epochs=15, batch_size=32, validation_data=([(X_val_img, X_val_text)], tf.convert_to_tensor(y_val)))

Epoch 1/15
98/98 [==============================] - 83s 404ms/step - loss: 0.3451 - accuracy: 0.2028 - auc: 0.7412 - binary_accuracy: 0.8748 - categorical_accuracy: 0.2028 - val_loss: 0.3404 - val_accuracy: 0.2206 - val_auc: 0.7287 - val_binary_accuracy: 0.8779 - val_categorical_accuracy: 0.2206
Epoch 2/15
98/98 [==============================] - 33s 338ms/step - loss: 0.3197 - accuracy: 0.2216 - auc: 0.7792 - binary_accuracy: 0.8811 - categorical_accuracy: 0.2216 - val_loss: 0.3393 - val_accuracy: 0.2193 - val_auc: 0.7514 - val_binary_accuracy: 0.8761 - val_categorical_accuracy: 0.2193
Epoch 3/15
98/98 [==============================] - 32s 326ms/step - loss: 0.2959 - accuracy: 0.2462 - auc: 0.8208 - binary_accuracy: 0.8863 - categorical_accuracy: 0.2462 - val_loss: 0.3714 - val_accuracy: 0.2069 - val_auc: 0.7244 - val_binary_accuracy: 0.8691 - val_categorical_accuracy: 0.2069
Epoch 4/15
98/98 [==============================] - 31s 319ms/step - loss: 0.2601 - accuracy: 0.2688 - auc: 0

In [ ]:
multimodal_model.evaluate([X_test_img, X_test_text], y_test)

fatal: not a git repository (or any of the parent directories): .git
